# Dataset processing

In [2]:
%load_ext watermark
%matplotlib inline

import glob

import cf_xarray as cfxr
import dcpy
import distributed
import eddydiff as ed
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from IPython.display import Image

xr.set_options(keep_attrs=True)

plt.rcParams["figure.dpi"] = 140
plt.rcParams["savefig.dpi"] = 200
plt.style.use("ggplot")

%watermark -iv

cf_xarray  : 0.8.1.dev0+gce9cd5a.d20230327
matplotlib : 3.7.1
numpy      : 1.23.5
eddydiff   : 0.1
xarray     : 2023.3.0
dcpy       : 0.2.1.dev45+g67ec22a.d20230504
distributed: 2023.3.2



## POP 1° model output

#### POP 1/10° state vars climatology

In [ ]:
pop_hires_clim = (
    xr.load_dataset("../datasets/pop-hires-natre-coarsened-annual-climatology.nc")
    .pint.quantify()
    .squeeze("cycle")
    .map(dcpy.util.to_base_units)
)
# pop_hires_clim['z_σ'] = pop_hires_clim.z_σ.cf.ffill("Z")
# pop_hires_clim = pop_hires_clim.cf.add_bounds("z_σ", dim="Z").isel(σ=slice(-7))
pop_hires_clim["TEMP"].attrs["standard_name"] = "sea_water_potential_temperature"
# NOTE: This only adds delT2, and averages in box
pop_hires_clim_profile = ed.pop.calc_mean_redivar_profile(pop_hires_clim)

# interpolate to pop_natre Z grid so we can estimate Ke
pop_hires_clim_profile = (
    pop_hires_clim_profile.swap_dims({"σ": "z_σ"})
    .drop_vars("z_σ_bounds")
    .dropna("z_σ")
    .interp(z_σ=pop_hires_natre.depth)
    .cf.add_bounds("depth")
)
pop_hires_clim_profile["Ke"] = (
    -1 * pop_hires_natre.DISS
) / pop_hires_clim_profile.delT2_plane.where(pop_hires_clim_profile.delT2_plane > 5e-13)

pop_hires_clim_profile

#### POP 1° spinup

In [22]:
%autoreload

ds_ = xr.load_dataset(
    "../datasets/pop-1deg-redi-var-natre-cycle-0.zarr/",
    engine="zarr",
)
ds_.z_σ.attrs["units"] = "cm"
ds_.σ.attrs.update({"axis": "Z", "positive": "down"})
ds_.TEMP.attrs["standard_name"] = "sea_water_potential_temperature"

month1 = ds_.isel(yearmonth=0).pint.quantify().map(dcpy.util.to_base_units).squeeze()
month1_profile = ed.pop.calc_mean_redivar_profile(month1)
month1_profile = month1_profile.where(month1_profile.z_σ.notnull(), drop=True).isel(
    σ=slice(1, None)
)
month1_profile.pint.dequantify().to_netcdf("../datasets/release/pop-1-month1.nc")

pop_1deg_spinup = (
    ds_.squeeze("cycle")
    .coarsen(yearmonth=120, boundary="trim")
    .mean()
    .rename({"yearmonth": "decade"})
    .pint.quantify()
    .map(dcpy.util.to_base_units)
)
pop_1deg_spinup_profile = ed.pop.calc_mean_redivar_profile(pop_1deg_spinup)
pop_1deg_spinup_profile.to_netcdf("../datasets/release/pop-1-spinup-natre.nc")
pop_1deg_spinup_profile

/Users/dcherian/mambaforge/envs/eddydiff/lib/python3.9/site-packages/xarray/core/duck_array_ops.py:675: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return push(array, n, axis)
/Users/dcherian/mambaforge/envs/eddydiff/lib/python3.9/site-packages/xarray/core/duck_array_ops.py:675: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return push(array, n, axis)
/var/folders/yp/rzgd0f7n5zbcbf5dg73sfb8ntv3xxm/T/ipykernel_51954/879300290.py:27: SerializationWarning: saving variable month with floating point data as an integer dtype without any _FillValue to use for NaNs
  pop_1deg_spinup_profile.to_netcdf("../datasets/release/pop-1-spinup-natre.nc")
/var/folders/yp/rzgd0f7n5zbcbf5dg73sfb8ntv3xxm/T/ipykernel_51954/879300290.py:27: SerializationWarning: saving variable year with floating point data as an integer dtype without any _FillValue to use for NaNs
  pop_1deg_spinup_profile.to_netcdf("../datasets/release/p

<xarray.Dataset>
Dimensions:      (decade: 6, σ: 60, bounds: 2)
Coordinates:
    cycle        int64 0
    month        (decade) float64 6.5 6.5 6.5 6.5 6.5 6.5
    time         (decade) object 0005-12-16 00:00:00 ... 0055-12-16 00:00:00
    year         (decade) float64 5.5 15.5 25.5 35.5 45.5 55.5
  * σ            (σ) float32 34.15 34.15 34.17 34.18 ... 37.2 37.2 37.2 37.2
    z_σ          (decade, σ) float64 66.59 66.93 67.3 ... 4.931e+03 4.931e+03
    z_σ_bounds   (decade, σ, bounds) float64 66.42 66.76 ... 4.931e+03 4.931e+03
Dimensions without coordinates: decade, bounds
Data variables:
    KAPPA_ISOP   (decade, σ) float64 [m²/s] 2.512e+03 2.518e+03 ... nan nan
    RediVar      (decade, σ) float64 [K²/s] 3.03e-09 3.025e-09 ... nan nan
    TEMP         (decade, σ) float64 [K] 295.8 295.8 295.7 295.7 ... nan nan nan
    delT2        (decade, σ) float64 [K²/m²] 1.186e-12 1.183e-12 ... nan nan
    delT2_plane  (decade, σ) float64 4.255e-13 4.351e-13 4.502e-13 ... 0.0 0.0
Attributes:
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    calendar:          All years have exactly  365 days.
    cell_methods:      cell_methods = time: mean ==> the variable values are ...
    contents:          Diagnostic and Prognostic Variables
    history:           none
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    revision:          $Id$
    source:            CCSM POP2, the CCSM Ocean Component
    start_time:        This dataset was created on 2019-06-20 at 13:53:17.7
    time_period_freq:  month_1
    title:             g.e21.GOMIPECOIAF_JRA.TL319_g17.CMIP6-omip2.001

## Cole et al (2015)


Use 1° file downloaded from 

In [7]:
cole = ed.read_cole()

cole.to_netcdf("../datasets/release/cole2015.nc")

## Groeskamp et al (2020)

Use MATLAB files from

In [8]:
import os

import h5py
from scipy.io import loadmat

file = h5py.File(os.path.expanduser("~/datasets/groeskamp2020/WOA18_K.mat"))

file0 = h5py.File(os.path.expanduser("~/datasets/groeskamp2020/WOA18_K0.mat"))

tuple(file0["WOA18_K0"].keys())

('K0_WOA',)

In [9]:
ds = xr.Dataset(
    coords={
        "depth": (
            "depth",
            np.array(file["WOA18_K"]["z"]).squeeze(),
            {"standard_name": "depth", "units": "m", "positive": "down"},
        ),
        "lat": (
            "lat",
            np.array(file["WOA18_K"]["y"]).squeeze(),
            {"standard_name": "latitude", "units": "degrees_north"},
        ),
        "lon": (
            "lon",
            np.array(file["WOA18_K"]["x"]).squeeze(),
            {"standard_name": "longitude", "units": "degrees_east"},
        ),
    }
)

ds["Ke_0"] = (
    ("depth", "lat", "lon"),
    file0["WOA18_K0"]["K0_WOA"],
    {"description": "unsuppressed diffusivities", "long_name": "$K_e$"},
)
ds["Ke"] = (("depth", "lat", "lon"), file["WOA18_K"]["K_WOA"], {"long_name": "$K_e$"})
ds["S"] = (
    ("depth", "lat", "lon"),
    file["WOA18_K"]["S_WOA"],
    {"long_name": "$S$", "description": "suppression factor"},
)

In [10]:
ds

<xarray.Dataset>
Dimensions:  (depth: 102, lat: 180, lon: 360)
Coordinates:
  * depth    (depth) float64 0.0 5.0 10.0 15.0 ... 5.3e+03 5.4e+03 5.5e+03
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
Data variables:
    Ke_0     (depth, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    Ke       (depth, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    S        (depth, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan

In [11]:
ds.to_netcdf(os.path.expanduser("~/datasets/groeskamp2020/groeskamp2020.nc"))